In [ ]:
!pip install koreanize-matplotlib #한글폰트 라이브러리 설치
!apt-get install -y fonts-nanum #나눔 고딕 폰트 설치
!pip install -q gdown

import gdown
import matplotlib.pyplot as plt
import koreanize_matplotlib  # 한글 자동 설정됨
import pandas as pd
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

# plt.rcParams['font.family'] = 'NanumGothic' #나눔 고딕 폰트 설정

In [ ]:
# CSV 파일 불러오기

transactions_file_id = '1rzQ8Bz6hGQ7IYZPWzEkaXIJXvA6pCFkq'
output_path = 'transactions_hm.csv'
gdown.download(f'https://drive.google.com/uc?id={transactions_file_id}', output_path, quiet=False)


articles = pd.read_csv('/content/drive/MyDrive/h&m dataset/h&m dataset/articles_hm.csv')
customers = pd.read_csv('/content/drive/MyDrive/h&m dataset/h&m dataset/customer_hm.csv')
transactions = pd.read_csv('/content/drive/MyDrive/h&m dataset/h&m dataset/transactions_hm.csv')

In [ ]:
# 각 테이블에 대한 열과 행 개수를 표현해주세요.
articles.shape # (105542, 25)
customers.shape # (1048575, 6)
transactions.shape # (1048575, 5)

In [ ]:
# 각 테이블에 대한 컬럼타입과 통계량을 보여주세요.
articles.info()
articles.describe()
customers.info()
customers.describe()
transactions.info()
transactions.describe()

In [ ]:
# NULL 값이 존재하는 경우, 대체 or 제거 모두 가능합니다.
customers.isnull().sum() # fashion_news_frequency (패션 뉴스 알람 주기) 컬럼 null 값 1개
articles.isnull().sum() # detail_desc (제품 상세 설명) 컬럼 null 값 416
transactions.isnull().sum() # null 값 0개

# NULL 값 제거/대체 필요없어 보임

In [ ]:
customer.isnull().sum()

## **연령대 별 매출 현황**

In [ ]:
# 연령대 컬럼 생성
def get_age_group(age):
  if age < 20:
    return '10대 이하'
  elif age < 30:
    return '20대'
  elif age < 40:
    return '30대'
  elif age < 50:
    return '40대'
  elif age < 60:
    return '50대'
  else:
    return '60대 이상'

customers['age_group'] = customers['age'].apply(get_age_group)

In [ ]:
# 테이블을 결합하여, 데이터분석을 위한 하나의 데이터셋으로 만들어주세요.
# 모든 테이블을 결합하지 않아도 좋습니다.
# 테이블 결합 시 inner join 으로 진행해주세요.

# 거래 데이터에 고객 정보 병합
merged_df = pd.merge(transactions, customers, on='customer_id', how='inner')

In [ ]:
merged_df.head()

In [ ]:
# 연령대별 매출 집계
sales_by_age = merged_df.groupby('age_group')['price'].sum().reset_index()

sales_by_age

In [ ]:
plt.figure(figsize=(8,5))
plt.bar(sales_by_age['age_group'], sales_by_age['price'])
plt.title('연령대 별 매출 현황')
plt.xlabel('연령대')
plt.ylabel('총 매출')
plt.show()

In [ ]:
'''
분석 요약
1. 20대가 가장 큰 매출 비중을 차지하고 있다.
2. 그 다음은 30대, 50대, 40대가 많이 차지
3. 10대 이하는 매출 비중이 가장 낮다

'''

## **채널별 판매 비중**

In [ ]:
# 연령대 컬럼 생성
def get_age_group(age):
    if age < 20:
        return '10대 이하'
    elif age < 30:
        return '20대'
    elif age < 40:
        return '30대'
    elif age < 50:
        return '40대'
    elif age < 60:
        return '50대'
    else:
        return '60대 이상'


In [ ]:
customers['age_group'] = customers['age'].apply(get_age_group)

In [ ]:
customers.head()

In [ ]:
# 거래 데이터와 고객 데이터를 customer_id 기준으로 INNER JOIN
merged = pd.merge(transactions, customers, on='customer_id', how='inner')

In [ ]:
# 전체 고객 대상 온라인/오프라인 구매 비중
total_counts = merged['sales_channel_id'].value_counts().sort_index()
total_ratio = total_counts / total_counts.sum() * 100


print("전체 고객 대상 온라인/오프라인 구매 비중 (%)")
print(total_ratio)
print(total_counts)

# 온라인 비중이 더 높다

In [ ]:
# 파이 차트로 보기

ratio = total_ratio
labels = ['오프라인', '온라인']
explode = [0, 0.01]
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(ratio, labels=labels, autopct='%.1f%%', explode=explode, wedgeprops=wedgeprops)
plt.title('채널별 구매 비율')
plt.show()

In [ ]:
# 연령대별 온라인/오프라인 구매 비중
age_channel_counts = merged.groupby(['age_group', 'sales_channel_id']).size().unstack(fill_value=0)
age_channel_ratio = age_channel_counts.div(age_channel_counts.sum(axis=1), axis=0) * 100

print("\n연령대별 온라인/오프라인 구매 비중 (%)")
print(age_channel_ratio)

In [ ]:
''''''

# age_channel_ratio 재정리 (인덱스 초기화)
plot_df = age_channel_counts.reset_index()

# 컬럼명 매핑 (1 → 오프라인, 2 → 온라인)
plot_df = plot_df.rename(columns={1: '오프라인', 2: '온라인'})

# Melt로 긴 형태로 변환 (seaborn barplot에 적합한 형태로 변환)
plot_df_melt = pd.melt(plot_df, id_vars='age_group', value_vars=['오프라인', '온라인'],
                       var_name='구매채널', value_name='비중')

# 시각화
plt.figure(figsize=(10,6))
sns.barplot(x='age_group', y='비중', hue='구매채널', data=plot_df_melt)
plt.title('연령대별 온라인/오프라인 매출 비교')
plt.ylabel('매출')
plt.xlabel('연령대')
plt.legend(title='구매채널')

plt.show()

In [ ]:
# age_channel_ratio 재정리 (인덱스 초기화)
plot_df = age_channel_ratio.reset_index()

# 컬럼명 매핑 (1 → 오프라인, 2 → 온라인)
plot_df = plot_df.rename(columns={1: '오프라인', 2: '온라인'})

# Melt로 긴 형태로 변환 (seaborn barplot에 적합한 형태로 변환)
plot_df_melt = pd.melt(plot_df, id_vars='age_group', value_vars=['오프라인', '온라인'],
                       var_name='구매채널', value_name='비중')

# 시각화
plt.figure(figsize=(10,6))
sns.barplot(x='age_group', y='비중', hue='구매채널', data=plot_df_melt)
plt.title('연령대별 온라인/오프라인 구매 비율(%)')
plt.ylabel('비중 (%)')
plt.xlabel('연령대')
plt.legend(title='구매채널')
plt.ylim(0, 100)
plt.show()